### Prøv

- Finn ut hvordan alternativt definere cycle life. Kapasitet i en c20 syklus rundt et visst antall?
- Bruk excel-ark og prøv å finne så store grupper som mulig med betingelser
- Finn c20-sykluser i nærheten av hverandre
- Begynn å tenke på vanlige cycle data, hvordan plukke ut riktige sykluser, gjennomsnitt av et intervall? Må vel gjøres i makeinterimdataife

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import os, random
import pathlib
import glob

pd.set_option('display.max_columns', 100)

from sklearn.pipeline import Pipeline
from sklearn.linear_model import ElasticNetCV
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, StratifiedKFold
from functions import get_errors
from sklearn.metrics import mean_squared_error, r2_score

In [2]:
# Make a nested dictionary to easily access all data. first keys are cell-ids, second keys are charge, discharge or summary data
# Only c20 cycles are included at this point
cell_dict = {}

path_cells = r"C:\Users\ife12216\OneDrive - Institutt for Energiteknikk\Documents\Masteroppgave\ML_github\data\IFE\interim"

# Loop through cells
for cell in os.listdir(path_cells)[1:]: # Exclude Cell_ID_MSB19108E0
    
    # Make summary dataframes
    cell_dir = os.path.join(path_cells, cell)
    summary_file = os.path.join(cell_dir, os.listdir(cell_dir)[1])
    summary_df = pd.read_csv(summary_file, encoding = "ISO-8859-1", sep='\t')
    
    charge_cycles = []
    discharge_cycles = []
    cell_dict[cell] = {} # each cell is a new dictionary
    
    c20_dir = os.path.join(path_cells, cell+'\C20_cycles')
    
    # Loop through files with charge cycles and add them to list
    for charge_cycle in glob.glob(os.path.join(c20_dir, '*cha.txt')):
        charge_cycle_file = os.path.join(c20_dir, charge_cycle)
        charge_cycle_df = pd.read_csv(charge_cycle_file, encoding = "ISO-8859-1", sep='\t')
        charge_cycles.append(charge_cycle_df)
    
    # Loop through files with discharge cycles and add them to list
    for discharge_cycle in glob.glob(os.path.join(c20_dir, '*dch.txt')):
        discharge_cycle_file = os.path.join(c20_dir, discharge_cycle)
        discharge_cycle_df = pd.read_csv(discharge_cycle_file, encoding = "ISO-8859-1", sep='\t')
        discharge_cycles.append(discharge_cycle_df)

    # Add summary, charge cycles and discharge cycles to cell dictionary
    cell_dict[cell]['summary'] = summary_df
    cell_dict[cell]['charge'] = charge_cycles
    cell_dict[cell]['discharge'] = discharge_cycles

In [14]:
for cell in cell_dict.keys():
    summary = cell_dict[cell]['summary']
    c20 = summary.loc[summary['Current_dch (C)'] == -0.05]
    #print(c20['Cycle - total - normalized'].between(200, 300).values)
    #print(list(c20['Cycle - total - normalized'].values)[3])
    c20.query('200 <= Cycle - total - normalized <= 400')

UndefinedVariableError: name 'Cycle' is not defined